In [1]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
root_img = 'data\JPEGImages'
root_xml = 'data\Annotations'

In [3]:
full_img_path = [ os.path.join(root_img, path) for path in os.listdir(root_img)]
full_xml_path = [ os.path.join(root_xml, path) for path in os.listdir(root_xml)]

In [4]:
xmls = []
for path in full_xml_path:
    tree = ET.parse(path)
    root = tree.getroot()
    xmls.append(root)

print(f'Number of xmls : {len(xmls)}')


Number of xmls : 2682


In [5]:
labels =[]
for root in xmls:
    name =[]
    for obj in root.findall(".//object"):
        name.append(obj.find("name").text.strip().lower())
    labels.append(name)

In [15]:
with open(r'data\labels.txt') as f:
    lbl = [(l.strip().strip(',')) for l in f]

print(lbl)

['sunglass', 'hat', 'jacket', 'shirt', 'pants', 'shorts', 'skirt', 'dress', 'bag', 'shoe']


In [16]:
idx = 0
dic ={}
for l in lbl:
    dic[l] = idx
    idx +=1

In [17]:
dic

{'sunglass': 0,
 'hat': 1,
 'jacket': 2,
 'shirt': 3,
 'pants': 4,
 'shorts': 5,
 'skirt': 6,
 'dress': 7,
 'bag': 8,
 'shoe': 9}

In [10]:
mapped_data = [[dic[label] for label in sublist] for sublist in labels]

print(mapped_data)

[[8, 7, 9], [3, 8, 9, 6], [3, 9, 6], [7, 1, 9], [3, 9, 6], [8, 3, 9, 6], [3, 9, 6], [8, 3, 9], [3, 8, 9, 5, 0], [3, 4, 9, 0], [8, 3, 9, 5], [3, 8, 3, 9, 6], [3, 1, 4, 9, 0], [3, 4, 9], [3, 8, 2, 9, 6], [3, 8, 2, 9, 6], [8, 2, 9, 5], [8, 7, 9, 2], [3, 8, 4, 9], [3, 9, 5, 0], [3, 2], [8, 3, 9, 5, 0], [7, 9], [3, 8, 2, 9, 5, 0], [3, 8, 1, 9, 6], [8, 7, 9, 0], [7, 9], [3, 9, 6], [3, 4, 2], [8, 2, 7, 9], [7, 9], [3, 9, 6], [8, 3, 4, 9, 2], [3, 8, 2, 4, 9], [3, 8, 2, 9, 5], [3, 4, 9], [8, 7, 9, 0], [3, 4, 9, 2], [8, 4, 9, 2], [8, 3, 2, 4, 9, 0], [3, 9, 5, 0], [8, 7, 1, 9], [3, 9, 5], [3, 4, 9], [8, 7, 9, 0], [2, 7, 9], [8, 3, 9, 5], [3, 8, 9, 6], [3, 9, 5], [3, 8, 3, 4, 9], [3, 2, 4, 9, 6], [3, 9, 6], [8, 3, 4, 9, 2], [8, 3, 4, 9, 2], [3, 8, 4, 9, 0], [3, 9, 5], [3, 2, 4, 9], [8, 7, 9], [3, 9, 6, 0], [8, 4, 9, 2], [3, 9, 6], [8, 7, 9], [1, 9, 5, 2], [8, 2, 3, 4, 9], [3, 1, 9, 6], [8, 7, 9], [3, 9, 6, 0], [3, 9, 6, 0], [2, 3, 4, 9], [8, 2, 7, 9], [3, 8, 2, 4, 9], [3, 8, 1, 9, 5], [3, 8, 4, 9,

In [11]:
num_classes = 10 
multi_hot_labels = []

for label_list in mapped_data:
    vec = np.zeros(num_classes, dtype=int)
    vec[label_list] = 1 
    multi_hot_labels.append(vec)

multi_hot_labels = np.array(multi_hot_labels)
print(multi_hot_labels.shape)  


(2682, 10)


In [14]:
np.save('data/labels.npy', multi_hot_labels)

In [ ]:
'''
shirt_shapes = []
jacket_shapes = []
pant_shapes = []
shoe_shapes = []

for root in xmls:
    exists = {"shirt": False, "jacket": False, "pants": False, "shoe": False}
    
    for obj in root.findall(".//object"):
        name = obj.find("name").text.strip().lower()
        bbox = obj.find("bndbox")

        if bbox is not None:
            xmin = int(bbox.find("xmin").text) if bbox.find("xmin") is not None else 0
            ymin = int(bbox.find("ymin").text) if bbox.find("ymin") is not None else 0
            xmax = int(bbox.find("xmax").text) if bbox.find("xmax") is not None else 0
            ymax = int(bbox.find("ymax").text) if bbox.find("ymax") is not None else 0
            shape = [xmin, ymin, xmax, ymax]
        else:
            shape = [0, 0, 0, 0]

        if name == "shirt":
            shirt_shapes.append(shape)
            exists["shirt"] = True
        elif name == "jacket":
            jacket_shapes.append(shape)
            exists["jacket"] = True
        elif name == "pants":
            pant_shapes.append(shape)
            exists["pants"] = True
        elif name == "shoe":
            shoe_shapes.append(shape)
            exists["shoe"] = True

    if not exists["shirt"]:
        shirt_shapes.append([0, 0, 0, 0])
    if not exists["jacket"]:
        jacket_shapes.append([0, 0, 0, 0])
    if not exists["pants"]:
        pant_shapes.append([0, 0, 0, 0])
    if not exists["shoe"]:
        shoe_shapes.append([0, 0, 0, 0])
'''

In [10]:
shirt_shapes = []
jacket_shapes = []
pant_shapes = []
shoe_shapes = []

for root in xmls:
    shapes = {"shirt": [0,0,0,0], "jacket": [0,0,0,0], "pants": [0,0,0,0], "shoe": [0,0,0,0]}
    
    for obj in root.findall(".//object"):
        name = obj.find("name").text.strip().lower()
        if name not in shapes:  # ignore unexpected labels
            continue

        # only take the first object of each type per XML
        if shapes[name] == [0,0,0,0]:
            bbox = obj.find("bndbox")
            if bbox is not None:
                xmin = int(bbox.find("xmin").text) if bbox.find("xmin") is not None else 0
                ymin = int(bbox.find("ymin").text) if bbox.find("ymin") is not None else 0
                xmax = int(bbox.find("xmax").text) if bbox.find("xmax") is not None else 0
                ymax = int(bbox.find("ymax").text) if bbox.find("ymax") is not None else 0
                shapes[name] = [xmin, ymin, xmax, ymax]
    
    shirt_shapes.append(shapes["shirt"])
    jacket_shapes.append(shapes["jacket"])
    pant_shapes.append(shapes["pants"])
    shoe_shapes.append(shapes["shoe"])


In [12]:
print(f'Length of shirt : {len(shirt_shapes)}\nJacket length : {len(jacket_shapes)}\nPnats length : {len(pant_shapes)}\nShoes length : {len(shoe_shapes)}')

Length of shirt : 2682
Jacket length : 2682
Pnats length : 2682
Shoes length : 2682


In [14]:
images=[]
for path in full_img_path:
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    images.append(image)
len(images)

[98, 136, 206, 301]

In [25]:
images = []
shirt_crops = []
jacket_crops = []
pant_crops = []
shoe_crops = []

def crop_and_resize(image, bbox, target_shape=(300, 200)):
    x1, y1, x2, y2 = bbox
    if x2 > x1 and y2 > y1:
        crop = image[y1:y2, x1:x2]
        crop = cv2.resize(crop, target_shape)
    else:
        crop = np.zeros((target_shape[1], target_shape[0], 3), dtype=np.uint8)  # blank image if bbox invalid
    return crop

for idx, path in enumerate(full_img_path):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    images.append(image)

    shirt_crops.append(crop_and_resize(image, shirt_shapes[idx]))
    jacket_crops.append(crop_and_resize(image, jacket_shapes[idx]))
    pant_crops.append(crop_and_resize(image, pant_shapes[idx]))
    shoe_crops.append(crop_and_resize(image, shoe_shapes[idx]))

print(len(images), len(shirt_crops), len(jacket_crops), len(pant_crops), len(shoe_crops))


2682 2682 2682 2682 2682


In [28]:
images = np.array(images)
shirts = np.array(shirt_crops)
jackets = np.array(jacket_crops)
pants = np.array(pant_crops)
shoes = np.array(shoe_crops)
print(f'Shirt : {shirts.shape}\nJackets : {jackets.shape}\nPants : {pants.shape}\nShoes : {shoes.shape}')

Shirt : (2682, 200, 300, 3)
Jackets : (2682, 200, 300, 3)
Pants : (2682, 200, 300, 3)
Shoes : (2682, 200, 300, 3)


In [30]:
np.save('data/images.npy', images)
np.save('data/shirts.npy', shirts)
np.save('data/jackets.npy', jackets)
np.save('data/pants.npy', pants)
np.save('data/shoes.npy', shoes)